In [7]:
import pandas as pd

In [8]:
import numpy as np

In [20]:
feu = pd.read_csv('combine.csv', sep=';')
feu['Date'] = pd.to_datetime(feu['Date de première alerte']).dt.date
feu['Feu'] = np.ones(len(feu))

# Filtrer les lignes où la valeur de la colonne "Département" est égale à 40
feu_landes = feu[feu['Département'] == '40']

# Afficher les premières lignes du DataFrame filtré
feu_landes

/tmp/ipykernel_2439/268027106.py:2: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  feu['Date'] = pd.to_datetime(feu['Date de première alerte']).dt.date


,Année,Numéro,Département,Code INSEE,Nom de la commune,Date de première alerte,Origine de l'alerte,Moyens de première intervention,Surface parcourue (m2),Surface forêt (m2),...,Nombre de bâtiments totalement détruits,Nombre de bâtiments partiellement détruits,Hygrométrie (%),Vitesse moyenne du vent (Km/h),Direction du vent,Température (°C),Précision de la donnée,Présence d'un contour valide,Date,Feu
652,2006,1876,40,40006,Arengosse,26/07/2006 19:40,Indéterminé,NaN,5000,5000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Non,",2006-07-26,1.0
653,2006,1877,40,40151,Lesgor,17/07/2006 15:29,Indéterminé,NaN,66000,60000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Non,",2006-07-17,1.0
654,2006,1878,40,40108,Gastes,25/04/2006 18:40,Indéterminé,NaN,400,400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Non,",2006-04-25,1.0
655,2006,1879,40,40210,Onesse-et-Laharie,17/07/2006 18:20,Indéterminé,NaN,59000,59000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Non,",2006-07-17,1.0
656,2006,1881,40,40167,Luxey,27/07/2006 10:49,Indéterminé,NaN,10,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Non,",2006-07-27,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43682,2022,10965,40,40162,Lucbardez-et-Bargues,17/05/2022 20:15,Indéterminé,NaN,8000,8000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Non,",2022-05-17,1.0
43683,2022,10967,40,40217,Parentis-en-Born,05/07/2022 16:12,Indéterminé,NaN,118000,8000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Non,",2022-07-05,1.0
43684,2022,10968,40,40267,Saint-Justin,03/06/2022 19:14,Indéterminé,NaN,100,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Non,",2022-06-03,1.0
43711,2022,11341,40,40168,Magescq,16/07/2022 16:05,Indéterminé,NaN,28000,28000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Non,",2022-07-16,1.0


In [23]:
# Compter le nombre de feux par jour
feux_par_jour = feu_landes.groupby('Date').size().reset_index(name='Nombre de feux')
feux_par_jour

,Date,Nombre de feux
0,2006-02-14,1
1,2006-03-16,1
2,2006-03-25,1
3,2006-03-27,1
4,2006-03-30,1
...,...,...
1035,2022-10-08,1
1036,2022-10-16,1
1037,2022-10-20,1
1038,2022-10-26,1


In [9]:
# Initialiser une liste vide pour stocker les données
data = []

# Ouvrir le fichier texte pour la lecture
with open('ifm donnees 2022 rcp4.5.txt', 'r') as file:
    for line in file:
        # Ignorer les lignes de commentaire
        if not line.startswith('#'):
            # Diviser la ligne pour obtenir les différentes valeurs
            columns = line.strip().split(';')
            # Ajouter les valeurs à la liste de données
            data.append(columns)

# Créer un DataFrame à partir de la liste de données
df_ifm45 = pd.DataFrame(data, columns=['Date', 'Latitude', 'Longitude', 'ifm', 'ifm-max'])

# Convertir les colonnes de données en types appropriés
df_ifm45['Date'] = pd.to_datetime(df_ifm45['Date'], format='%Y%m%d')
df_ifm45['Latitude'] = pd.to_numeric(df_ifm45['Latitude'])
df_ifm45['Longitude'] = pd.to_numeric(df_ifm45['Longitude'])
df_ifm45['ifm'] = pd.to_numeric(df_ifm45['ifm'])
df_ifm45['ifm-max'] = pd.to_numeric(df_ifm45['ifm-max'])

# Afficher les premières lignes du DataFrame pour vérification
print(df_ifm45.head())

        Date  Latitude  Longitude  ifm  ifm-max
0 2006-01-01   43.5074   -1.36829  0.7      0.8
1 2006-01-02   43.5074   -1.36829  0.6      0.7
2 2006-01-03   43.5074   -1.36829  0.6      0.7
3 2006-01-04   43.5074   -1.36829  0.9      0.9
4 2006-01-05   43.5074   -1.36829  0.9      1.0


In [25]:
# Fusion
df_ifm45['Date'] = pd.to_datetime(df_ifm45['Date']).dt.date
feu_data = pd.merge(df_ifm45, feux_par_jour, on='Date', how='inner')
feu_data

,Date,Latitude,Longitude,ifm,ifm-max,Nombre de feux
0,2006-02-14,43.5074,-1.36829,0.6,0.7,1
1,2006-03-16,43.5074,-1.36829,0.8,0.9,1
2,2006-03-25,43.5074,-1.36829,0.0,0.0,1
3,2006-03-27,43.5074,-1.36829,0.7,0.9,1
4,2006-03-30,43.5074,-1.36829,9.1,10.5,1
...,...,...,...,...,...,...
150795,2022-10-08,44.4544,-1.02851,0.0,0.0,1
150796,2022-10-16,44.4544,-1.02851,0.0,0.0,1
150797,2022-10-20,44.4544,-1.02851,1.7,1.7,1
150798,2022-10-26,44.4544,-1.02851,1.4,1.4,1


In [30]:
# Load data
data = feu_data

feu_data['Année'] = feu_data['Date'].dt.year
feu_data['Mois'] = feu_data['Date'].dt.month
feu_data['Jour'] = feu_data['Date'].dt.day

# Drop the original 'date' column
feu_data.drop(columns=['date'], inplace=True)

# Select features (FWI, latitude, longitude, date) and target variable (number of fires)
X = data[['ifm', 'Latitude', 'Longitude', 'Année','Mois','Jour']]
y = data['Nombre de feux']

AttributeError: Can only use .dt accessor with datetimelike values

In [27]:
from sklearn.linear_model import LogisticRegression

In [28]:
# Initialize logistic regression model
model = LogisticRegression()

# Train the model
model.fit(X, y)


TypeError: float() argument must be a string or a real number, not 'datetime.date'